In [135]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [136]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[0., 0., 0., 1.],
        [0., 1., 1., 1.],
        [0., 0., 0., 0.],
        [1., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [1., 0., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 0., 0., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 1.],
        [0., 1., 0., 1.],
        [0., 1., 0., 1.],
        [0., 0., 1., 1.],
        [1., 0., 1., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 0.],
        [1., 1., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 1.],
        [0., 0., 0., 1.],
        [1., 1., 0., 0.],
        [1., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 0.],
        [0., 1., 1., 1.],
        [1., 0., 0., 1.],
        [1., 1., 0., 0.],
        [1.,

Hamming(7,4) Encoder

In [137]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [138]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[0., 0., 0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 1., 1., 1., 0., 0.],
        [1., 0., 1., 0., 1., 0., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1., 0., 1., 1., 0., 1., 0.],
        [0.,

BPSK Modulator + Noise

In [139]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        data: data received from the Hamming(7,4) encoder(Tensor)
        symbol_rate: Symbol rate in Hz
        carrier_freq: Carrier frequency in Hz
        snr_dB: Signal-to-noise ratio in dB

    Returns:
        time:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, symbol_rate, carrier_freq, snr_dB):
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # Time vector
            time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
        
            # Generate carrier signal
            carrier = torch.cos(2 * torch.pi * carrier_freq * time)
        
            # Modulate the signal
            modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(-snr_dB / 10))
            noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
            noised_signal = modulated_signal + noise
            data[i] = noised_signal
    
       
        return time, data

In [140]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
time, modulated_noise_signal = modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[-0.8827,  0.4964,  0.7232,  0.5906, -0.7537, -0.5972,  1.0236],
        [-0.7987, -0.0810, -0.3209,  0.9059,  0.5022,  0.4287,  1.2094],
        [-0.9483,  0.5755,  0.5501, -0.7671,  0.4955,  0.4826, -0.9976],
        [ 0.9938,  0.5471, -0.7266, -0.8027, -0.7998,  0.1525,  0.8638],
        [-1.0663,  0.4223,  0.6286,  0.9091, -0.4625, -0.5821,  1.0886],
        [-0.9891,  0.5657,  0.1424, -1.2340,  0.3680,  0.6373, -0.8662],
        [ 0.6934,  0.2242,  0.4827,  1.3953,  0.9403,  0.4859,  1.1445],
        [ 1.0416,  0.4610, -0.3341,  0.7008,  0.8445, -0.5457, -0.9958],
        [ 1.0132, -0.5674,  0.6170, -0.9933,  0.2907, -0.5736,  0.8444],
        [-1.0914,  0.5416, -0.4055,  1.0905, -0.1583,  0.1635, -0.8897],
        [ 1.0795, -0.4579,  0.1993,  0.4445, -0.5880,  0.7793, -0.9635],
        [ 1.1160, -0.5191, -0.3674,  0.9393, -0.3261, -0.5959,  1.0605],
        [-0.9873, -0.7161, -0.4134,  1.1441,  0.5077,  0.5465,  0.8500],
        [-0.8346,  0.4033,  0.4770, -1.2263,  0.243

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [141]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal.
        noise_std (float): Standard deviation of the noise.

    Returns:
        torch.Tensor: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))
    # likelihood_0 = 2 * (signal - 1) / noise_std**2
    # likelihood_1 = 2 * (signal + 1) / noise_std**2

    # Calculate the LLR
    # llr_values = likelihood_0 / likelihood_1
    llr = 2 * signal / noise_std**2

    # return llr_values, llr
    return llr


In [142]:
# received_signal = torch.tensor([2.6811e-01, -7.7690e-01, -3.2614e-01,  1.6319e+00, -1.5100e-01, -3.3976e-01,  8.3065e-01])
# LLR values: tensor([0.0170, -0.0491, -0.0206,  0.1032, -0.0096, -0.0215,  0.0525])

snr_dB = 15

llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[-0.0558,  0.0314,  0.0457,  0.0374, -0.0477, -0.0378,  0.0647],
        [-0.0505, -0.0051, -0.0203,  0.0573,  0.0318,  0.0271,  0.0765],
        [-0.0600,  0.0364,  0.0348, -0.0485,  0.0313,  0.0305, -0.0631],
        [ 0.0629,  0.0346, -0.0460, -0.0508, -0.0506,  0.0096,  0.0546],
        [-0.0674,  0.0267,  0.0398,  0.0575, -0.0293, -0.0368,  0.0688],
        [-0.0626,  0.0358,  0.0090, -0.0780,  0.0233,  0.0403, -0.0548],
        [ 0.0439,  0.0142,  0.0305,  0.0882,  0.0595,  0.0307,  0.0724],
        [ 0.0659,  0.0292, -0.0211,  0.0443,  0.0534, -0.0345, -0.0630],
        [ 0.0641, -0.0359,  0.0390, -0.0628,  0.0184, -0.0363,  0.0534],
        [-0.0690,  0.0343, -0.0256,  0.0690, -0.0100,  0.0103, -0.0563],
        [ 0.0683, -0.0290,  0.0126,  0.0281, -0.0372,  0.0493, -0.0609],
        [ 0.0706, -0.0328, -0.0232,  0.0594, -0.0206, -0.0377,  0.0671],
        [-0.0624, -0.0453, -0.0261,  0.0724,  0.0321,  0.0346,  0.0538],
        [-0.0528,  0.0255,  0.0302, -0.

LDPC Decoder

In [146]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H, max_iter=50):
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.max_iter = max_iter
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr):
        for iteration in range(self.max_iter):
            # Variable node to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product0 = 0.5 * self.messages_v_to_c[connected_checks, j]
                    product = torch.prod(torch.tanh(product0))
                    # product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, i]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check node to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msg0 = self.messages_c_to_v[connected_vars, i]
                    sum_msgs = torch.sum(sum_msg0) - self.messages_v_to_c[j, i]
                    # sum_msgs = torch.sum(self.messages_v_to_c[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))

        return estimated_bits
    
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])

llr_demodulator_output = llr_output
ldpc_bp = LDPCBeliefPropagation(H)
estimated_bits = ldpc_bp(llr_demodulator_output)

print("LLR Demodulator Output:", llr_demodulator_output)
print("Estimated Bits:", estimated_bits)

RuntimeError: expand(torch.FloatTensor{[7]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [130]:
# class LDPCDecoder(nn.Module):
#     def __init__(self, H, num_iterations=10):
#         super(LDPCDecoder, self).__init__()
#         self.H = nn.Parameter(H, requires_grad=False)  # Parity-check matrix
#         self.num_iterations = num_iterations
#         
#     def ldpc_decode(self, llr_input):
#         # Your LDPC decoding algorithm (e.g., SPA or BP)
#         # ...
# 
#         return
# 
#     def forward(self, llr_input):
#         # Implement LDPC decoding algorithm
#         decoded = self.ldpc_decode(llr_input)
#         return decoded
# 
# 
# 
# def ldpc_loss(llr_hd, true):
#     
#     return 
#     
# 
# num_epochs = 20
# 
# # Set up your dataset (LLR values and corresponding codewords)
# # Initialize the LDPCDecoder model
# ldpc_decoder = LDPCDecoder(H)
# 
# # Define optimizer and learning rate
# optimizer = optim.Adam(ldpc_decoder.parameters(), lr=0.001)
# 
# # Training loop
# for epoch in range(num_epochs):
#     for llr_input, true_codeword in dataloader:
#         optimizer.zero_grad()
#         decoded = ldpc_decoder(llr_input)
#         loss = ldpc_loss(decoded, true_codeword) #cross entropy
#         loss.backward()
#         optimizer.step()
# 
#     # Print or log the loss for monitoring training progress
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


IndentationError: expected an indented block (2525148438.py, line 18)